# Generate FIR Filter Coefficients

Demonstrates how to generate precomputed/predefined FIR filter coefficients for various resampling scenarios based on input and output sample rates. The coefficients are formatted in a convenient way to facilitate adding them to a C/C++ file as compile-time constants. 

In [117]:
from fractions import Fraction
import math
import numpy as np
import os
from scipy import signal

In [118]:
INPUT_SAMPLE_RATES_STR = os.environ.get('INPUT_SAMPLE_RATE', '1140000,1140000,1000000,228000,200000,200000')
OUTPUT_SAMPLE_RATES_STR = os.environ.get('OUTPUT_SAMPLE_RATE', '228000,200000,200000,45600,45000,44000')
FILTER_ORDERS_STR = os.environ.get('FILTER_ORDERS', '32,64,128,256')

# convert from the string environment variables to other, more convenient, representations
INPUT_SAMPLE_RATES = [int(x) for x in INPUT_SAMPLE_RATES_STR.split(',')]
OUTPUT_SAMPLE_RATES = [int(x) for x in OUTPUT_SAMPLE_RATES_STR.split(',')]
FILTER_ORDERS = [int(x) for x in FILTER_ORDERS_STR.split(',')]

In [132]:
INPUT_TEMPLATE_FILE = "../../src/transform/falcon_dsp_predefined_fir_filter_template.cc"
OUTPUT_FILE = "../../src/transform/falcon_dsp_predefined_fir_filter.cc"

input_fd = open(INPUT_TEMPLATE_FILE, 'r')
output_fd = open(OUTPUT_FILE, 'w')

# write the first part of the file
for line in input_fd:
    if "AUTO_GENERATED_COEFFICIENTS_HERE" in line:
        break
    else:
        output_fd.write(line)

In [133]:
def write_filter_info(output_fd, filter_order, filter_order_str, filter_type, filter_coeffs):
    
    taps_enum_dict = {"-1": "FILTER_TAPS_OPTIMAL",
                      "16": "FILTER_TAPS_16",
                      "32": "FILTER_TAPS_32",
                      "64": "FILTER_TAPS_64",
                      "128": "FILTER_TAPS_128",
                      "256": "FILTER_TAPS_256"}
    
    # print 'header information'
    output_fd.write("        /* INPUT_SAMPLE_RATE:   %12u sps\n" % INPUT_SAMPLE_RATE)
    output_fd.write("         * OUTPUT_SAMPLE_RATE:  %12u sps\n" % OUTPUT_SAMPLE_RATE)
    output_fd.write("         * FILTER_ORDER:        %12u\n" % (filter_order))
    output_fd.write("         * COEFF_ALGORITHM:     %12s\n" % (filter_type))
    output_fd.write("         */\n")
    output_fd.write("        {\n");
    output_fd.write("            predefined_resample_filter_key_s(%u, %u, filter_taps_e::%s, filter_source_type_e::%s),\n" % (INPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE, taps_enum_dict[filter_order_str], filter_type))
    output_fd.write("            {%u, /* up_rate */\n" % p)
    output_fd.write("             %u, /* down_rate */\n" % q)
    output_fd.write("             std::vector<std::complex<float>>{\n")
    next_line = "                 "
    for coeff_idx in range(len(filter_coeffs)):
        next_line += "{%+01.08f, %01.08f}" % (filter_coeffs[coeff_idx].real, filter_coeffs[coeff_idx].imag)
        if coeff_idx % 4 == 3:
            output_fd.write(next_line + ",\n"); next_line = "                 "
        else:
            next_line += ", "

    output_fd.write(next_line[:-2])
    output_fd.write("             }\n")
    output_fd.write("            } /* end of %u sps -> %u sps */\n" % (INPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE))

In [134]:
DEFINED_FILTER_ORDERS = [16, 32, 64, 128, 256]

for resample_idx in range(len(INPUT_SAMPLE_RATES)):

    INPUT_SAMPLE_RATE = INPUT_SAMPLE_RATES[resample_idx]
    OUTPUT_SAMPLE_RATE = OUTPUT_SAMPLE_RATES[resample_idx]
    
    print("Generating coefficients for %u -> %u" % (INPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE))
    
    filter_orders = FILTER_ORDERS.copy()
    
    # find the 'unconstrained' filter order based on up and down sample ratios
    resampling_ratio = OUTPUT_SAMPLE_RATE / INPUT_SAMPLE_RATE
    ratio = Fraction("%.12f" % (resampling_ratio)).limit_denominator()
    p = ratio.numerator
    q = ratio.denominator
    pqmax = max(p, q)

    # cutoff frequency of the lowpass filter at the high (upsampled) rate
    cutoff_freq = 1 / 2 / pqmax
    unconstrained_filter_order = 2 * 10 * pqmax + 1
    #filter_delay = int((unconstrained_filter_order - 1) / p / 2)

    filter_orders.append(unconstrained_filter_order)
    filter_orders = sorted(list(set(filter_orders)))
    
    for filter_order_idx in range(len(filter_orders)):
    
        filter_order_str = ""
        if filter_orders[filter_order_idx] in DEFINED_FILTER_ORDERS:
            filter_order_str = "%u" % (filter_orders[filter_order_idx])
        else:
            filter_order_str = "-1"
    
        firls_num_taps = filter_orders[filter_order_idx]
        if firls_num_taps % 2 == 0:
            firls_num_taps += 1
        
        firls_filter_coeffs = float(p) * signal.firls(firls_num_taps, [0, 2.0 * cutoff_freq, 2.0 * cutoff_freq, 1.0], [1.0, 1.0, 0.0, 0.0])
        firls_filter_coeffs = firls_filter_coeffs * signal.kaiser(firls_num_taps, beta=5)
        write_filter_info(output_fd, firls_num_taps, filter_order_str, "FIRLS", firls_filter_coeffs)
        
        num_taps = filter_orders[filter_order_idx]
        if num_taps <= 200:
        
            try:
                remez_filter_coeffs = signal.remez(num_taps, [0, OUTPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE+(INPUT_SAMPLE_RATE/2-OUTPUT_SAMPLE_RATE)/4, INPUT_SAMPLE_RATE/2], [1,0], Hz=INPUT_SAMPLE_RATE, maxiter=100)
                output_fd.write("        },\n");
                write_filter_info(output_fd, num_taps, filter_order_str, "REMEZ", remez_filter_coeffs)
            except:
                print("Unable to generate REMEZ coefficients (%u -> %u, %u taps)" % (INPUT_SAMPLE_RATE, OUTPUT_SAMPLE_RATE, num_taps))
                pass
    
        if resample_idx != (len(INPUT_SAMPLE_RATES) - 1) or filter_order_idx != (len(filter_orders) - 1):
            output_fd.write("        },\n");
        else:
            output_fd.write("        }\n");

Generating coefficients for 1140000 -> 228000
Generating coefficients for 1140000 -> 200000
Generating coefficients for 1000000 -> 200000
Generating coefficients for 228000 -> 45600
Generating coefficients for 200000 -> 45000
Generating coefficients for 200000 -> 44000


In [135]:
# write the last part of the file
for line in input_fd:
    output_fd.write(line)
        
input_fd.close()
output_fd.close()